In [1]:
import pandas as pd
import numpy as np

In [2]:
#TourneyResults = pd.read_csv('DATA/NCAATourneyCompactResults.csv')
RegularResults = pd.read_csv('RegularSeasonCompactResults.csv')
RegularStat = pd.read_csv('DATA/RegularSeasonDetailedRollup_v2.csv')
TourneySeeds = pd.read_csv('NCAATourneySeeds.csv')
sampSub = pd.read_csv('SampleSubmissionStage2.csv')
#RegularSum = pd.read_csv('DATA/RegularSeasonDetailedRollup.csv')
RegularStatPG = pd.read_csv('RegularSeasonDetailedResults.csv')

In [3]:
Features = ['OffRt','DefRt','NetRating','AssRatio','TORatio','TrueShoot%','TotalR%','AvgPpg','AvgDR%','AvgBlk']
#possesions = 0.96 * (FGA + TO + (0.475 x FTA) - OR)
#offRt = 100 * (Score/Possesions)
#DefRt = 100 * (OpScore/OpPossesions)
#NetRating = 100 * (OffRt - DefRt)
#AssRatio = 100 * Assists / (FGA + (0.475 * FTA) + Assists + Turnovers))
#TORatio = 100 * TO / (FGA + (0.475 * FTA) + Assists + Turnovers)
#TrueShoot% = 100 * Pts / (2 * (FGA + (0.475 * FTA)))
#TotalR% = (DR + OR) / (DR + OR + OpDR + OpOR)
#AvgDR% = DR / (DR + OpDR)
requiredFeatures = ['Season','WTeamID','WScore','LTeamID','LScore','WFGA','WFTA','WAst','WDR','WOR','WTO','WBlk','LFGA','LFTA','LAst','LDR','LOR','LTO','LBlk']

In [4]:
Im = pd.DataFrame((RegularStatPG[requiredFeatures]))
#calculate needed stats for all games
Im['Wpossesions'] = 0.96 * (Im['WFGA']+Im['WTO']+0.475*Im['WFTA']-Im['WOR'])
Im['Lpossesions'] = 0.96 * (Im['LFGA']+Im['LTO']+0.475*Im['LFTA']-Im['LOR'])
Im['WOffRt'] = 100 * (Im['WScore'] / Im['Wpossesions'])
Im['WDefRt'] = 100 * (Im['LScore'] / Im['Lpossesions'])
Im['WNetRating'] = 100 * (Im['WOffRt'] - Im['WDefRt'])
Im['WAssRatio'] = 100 * (Im['WAst'] / (Im['WFGA'] + 0.475 * Im['WFTA'] + Im['WAst'] + Im['WTO']))
Im['WTORatio'] = 100 * (Im['WTO'] / (Im['WFGA'] + 0.475 * Im['WFTA'] + Im['WAst'] + Im['WTO']))
Im['WTrueShoot%'] = 100 * (Im['WScore'] / (2 * (Im['WFGA'] + 0.475 * Im['WFTA'])))
Im['WTotalR%'] = (Im['WDR'] + Im['WOR'])/(Im['WDR'] + Im['WOR'] + Im['LDR'] + Im['LOR'])
Im['WAvgDR%'] = Im['WDR'] / (Im['WDR'] + Im['LDR'])

Im['LOffRt'] = 100 * (Im['LScore'] / Im['Lpossesions'])
Im['LDefRt'] = 100 * (Im['WScore'] / Im['Wpossesions'])
Im['LNetRating'] = 100 * (Im['LOffRt'] - Im['LDefRt'])
Im['LAssRatio'] = 100 * (Im['LAst'] / (Im['LFGA'] + 0.475 * Im['LFTA'] + Im['LAst'] + Im['LTO']))
Im['LTORatio'] = 100 * (Im['LTO'] / (Im['LFGA'] + 0.475 * Im['LFTA'] + Im['LAst'] + Im['LTO']))
Im['LTrueShoot%'] = 100 * (Im['LScore'] / (2 * (Im['LFGA'] + 0.475 * Im['LFTA'])))
Im['LTotalR%'] = (Im['LDR'] + Im['LOR'])/(Im['LDR'] + Im['LOR'] + Im['WDR'] + Im['WOR'])
Im['LAvgDR%'] = Im['LDR'] / (Im['LDR'] + Im['WDR'])

In [5]:
#Take needed stats
Regular_Feat = Im[['Season','WTeamID','WScore','LTeamID','LScore','WScore','WBlk','WOffRt','WNetRating','WTrueShoot%','WTotalR%','WAvgDR%','LScore','LBlk','LOffRt','LNetRating','LTrueShoot%','LTotalR%','LAvgDR%']]

In [6]:
#create table of all games with date and matchups
SIDs = Regular_Feat[['Season','WTeamID','LTeamID']]
for i in range(0,len(SIDs['Season'])):
    if SIDs['WTeamID'][i] > SIDs['LTeamID'][i]:
        temp = SIDs['WTeamID'][i]
        SIDs['WTeamID'][i] = SIDs['LTeamID'][i]
        SIDs['LTeamID'][i] = temp

/Users/fraserthomson/ml/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3291: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
#Create Labels for each game
#calculate winning margin for each game(Score of team with lower id - Score of team with higher id)
TrainY = np.zeros(shape=(len(RegularStatPG['Season']),1))
for i in range(len(RegularStatPG['Season'])):
    marg = 0
    if int(RegularStatPG['WTeamID'][i]) > int(RegularStatPG['LTeamID'][i]):
        marg = RegularStatPG['LScore'][i] - RegularStatPG['WScore'][i]
    else:
        marg = RegularStatPG['WScore'][i] - RegularStatPG['LScore'][i]
    TrainY[i][0] = marg

In [8]:
#Rearrange stats so that stat of team with lower id is in front
X = np.zeros(shape=(len(RegularStatPG['Season']),14))
for i in range(0,len(TrainY)):
    if TrainY[i] > 0:
        X[i] = Regular_Feat.iloc[i][5:]
    elif TrainY[i] < 0:
        X[i] = pd.concat([Regular_Feat.iloc[i][12:],Regular_Feat.iloc[i][5:12]])

In [9]:
#Calculate season averages for each team for every year
TeamStats = pd.DataFrame(columns = ['Season','ID','Score','Blk','OffRt','NetRating','TrueShoot%','TotalR%','AvgDR%'],dtype=float)
TeamStats['Season'] = RegularStat['Season']+1
TeamStats['ID'] = RegularStat['TeamID']
TeamStats[['Score','Blk','OffRt','NetRating','TrueShoot%','TotalR%','AvgDR%']] = 0.0

In [10]:
TeamStats

,Season,ID,Score,Blk,OffRt,NetRating,TrueShoot%,TotalR%,AvgDR%
0,2015,1101,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016,1101,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017,1101,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018,1101,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019,1101,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2004,1102,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2005,1102,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2006,1102,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2007,1102,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2008,1102,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
Feat = ['Score','Blk','OffRt','NetRating','TrueShoot%','TotalR%','AvgDR%']
for i in range(0,len(TeamStats['Season'])):
    Season = TeamStats['Season'][i]
    ID = TeamStats['ID'][i]
    count = 0
    for j in range(0,len(SIDs['Season'])):
        if Season == int(SIDs['Season'][j]) and ID == int(SIDs['WTeamID'][j]):
            print(Season,ID)
            count += 1
            TeamStats['Score'][i] += X[j][0]
            TeamStats['Blk'][i] += X[j][1]
            TeamStats['OffRt'][i] += X[j][2]
            TeamStats['NetRating'][i] += X[j][3]
            TeamStats['TrueShoot%'][i] += X[j][4]
            TeamStats['TotalR%'][i] += X[j][5]
            TeamStats['AvgDR%'][i] += X[j][6]
        elif Season == int(SIDs['Season'][j]) and ID == int(SIDs['LTeamID'][j]):
            print(Season,ID)
            count += 1
            TeamStats['Score'][i] += X[j][7]
            TeamStats['Blk'][i] += X[j][8]
            TeamStats['OffRt'][i] += X[j][9]
            TeamStats['NetRating'][i] += X[j][10]
            TeamStats['TrueShoot%'][i] += X[j][11]
            TeamStats['TotalR%'][i] += X[j][12]
            TeamStats['AvgDR%'][i] += X[j][13]
    for feat in Feat:
        TeamStats[feat][i] /= count

2015 1101
2015 1101


/Users/fraserthomson/ml/env/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/fraserthomson/ml/env/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/fraserthomson/ml/env/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101
2015 1101


/Users/fraserthomson/ml/env/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2016 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2017 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2018 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101
2019 1101


...

2014 1457
2014 1457
2014 1457
2014 1457
2014 1457
2014 1457
2014 1457
2014 1457
2014 1457
2014 1457
2014 1457
2014 1457
2014 1457
2014 1457
2014 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2015 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2016 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457
2017 1457


2009 1459
2009 1459
2009 1459
2009 1459
2009 1459
2009 1459
2009 1459
2009 1459
2009 1459
2009 1459
2009 1459
2009 1459
2009 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2010 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2011 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459
2012 1459


2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2004 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2005 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2006 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461
2007 1461


2014 1462
2014 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2015 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2016 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462
2017 1462


2011 1464
2011 1464
2011 1464
2011 1464
2011 1464
2011 1464
2011 1464
2011 1464
2011 1464
2011 1464
2011 1464
2011 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2012 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2013 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2014 1464
2015 1464


In [16]:
#create test data with all possible matchups for 1 year with differneces in season averages for both teams as features 
matchups = np.zeros(shape=(2278,7))
Features = ['Blk','Score','OffRt','NetRating','TrueShoot%','TotalR%','AvgDR%']
year = 2019
for i in range(len(sampSub['ID'])):
    Team1 = int(sampSub['ID'][i][5:9])
    Team2 = int(sampSub['ID'][i][10:14])
    Team1Stats = np.zeros(shape = (7,1))
    Team2Stats = np.zeros(shape = (7,1))
    found1 = False
    found2 = False
    for j in range(len(TeamStats)):
        if int(TeamStats['Season'][j] == year) and int(TeamStats['ID'][j]) == Team1:
            for k in range(len(Features)):
                Team1Stats[k] = TeamStats[Features[k]][j]
                found1 = True
        elif int(TeamStats['Season'][j] == year) and int(TeamStats['ID'][j]) == Team2:
            for k in range(len(Features)):
                Team2Stats[k] = TeamStats[Features[k]][j]
                found2 = True
        if found1 and found2:
            break;
    
    print(i)
    feats = Team1Stats-Team2Stats
    for k in range(0,7):
        matchups[i][k] = feats[k]
    print(matchups[i])

0
[-6.67025862e-01 -6.08836207e+00  9.93101508e-01  4.37192866e+02
  1.28585421e+00 -3.46101706e-02 -3.57271655e-02]
1
[-2.21298174e+00 -7.15821501e+00 -5.76003467e+00 -2.35152980e+02
 -1.23680632e+00  6.53612874e-03  1.48188920e-02]
2
[-2.19827586e+00  6.78879310e-02  1.78707656e-02  4.07379418e+02
  2.09578133e+00 -5.10309359e-02 -4.27420839e-02]
3
[-1.24827586e+00 -1.51425287e+01 -9.20919092e+00 -5.79913657e+02
 -4.55904573e+00 -2.80106501e-02 -4.68163972e-02]
4
[-1.20585162e+00  5.36050157e+00  7.24498027e+00  9.37923578e+02
  2.74949979e+00 -1.09888847e-02 -1.76838043e-02]
5
[-1.53918495e+00 -1.30940439e+01 -5.71166276e+00 -7.82732764e+02
 -2.07274942e-01 -2.72996364e-02 -2.32252019e-02]
6
[-1.94827586e+00  1.82555781e-02 -1.74981736e+00 -2.31617339e+02
  2.92281221e+00 -4.45612100e-02 -3.58477221e-02]
7
[-8.42215256e-01 -3.79101358e+00 -3.60221705e+00  2.79475936e+02
 -2.95644984e+00 -3.44597364e-02 -4.99372364e-02]
8
[-4.27180527e+00 -1.17758621e+01 -5.67077132e+00 -1.04224983e+

73
[-1.75189394e-01  2.29734848e+00 -4.59531856e+00 -1.57716930e+02
 -4.24230405e+00  1.50434172e-04 -1.42100710e-02]
74
[-3.60477941e+00 -5.68750000e+00 -6.66387283e+00 -1.47944270e+03
 -1.76801176e+00 -9.67629542e-03 -2.22857773e-02]
75
[-4.26411290e-01  3.07056452e+00 -2.38011175e+00  2.34117156e+02
 -3.43687403e+00  4.04388070e-02  1.31612909e-02]
76
[-4.87132353e-01  9.54779412e+00  2.10956628e+00 -6.02762515e+01
  1.16203969e+00  3.01196770e-02  2.91387595e-02]
77
[-1.04595588e+00  2.90073529e+00  3.25591076e-01 -3.33161014e+02
 -4.10596482e-01 -4.71921148e-03 -4.41678369e-03]
78
[ 7.34879032e-01  2.03830645e+00 -2.12191145e+00  3.64366552e+02
 -3.52439972e+00  5.59103871e-02  1.57130346e-02]
79
[-1.46875000e+00  2.00000000e+00 -4.94087379e-01  2.33164835e+02
 -2.05234942e+00  7.52705611e-02  4.41466491e-02]
80
[-2.26609848e+00 -1.10359848e+01 -1.79256770e+01 -2.71308564e+03
 -8.02494730e+00 -1.24829462e-02 -4.66683859e-02]
81
[-1.28125000e+00  2.19485294e+00 -5.52954166e+00 -1.4

147
[ 1.98345588e+00  2.03860294e+00  5.20050812e+00  1.12018148e+03
  2.90321227e-01  2.49792781e-03 -2.66160503e-02]
148
[ 1.49197861e+00  5.79322638e-01  2.63735667e+00  7.10002502e+02
  1.85048295e-01 -1.72431236e-02 -3.65490899e-02]
149
[ 1.17647059e-01  1.44117647e+00  2.97742290e-01  4.71306400e+01
 -7.89171608e-01 -1.48435008e-02 -4.57384460e-02]
150
[ 8.23529412e-01  3.50000000e+00  7.47567195e+00  6.02148818e+02
  1.75882589e+00 -2.94263887e-02 -5.54586994e-02]
151
[ 2.49197861e+00  1.30641711e+01  1.19924928e+01  3.61116594e+02
  4.75578298e+00 -1.92353990e-02 -2.42782013e-02]
152
[-2.95900178e-01  2.15508021e+00 -2.82916870e-01 -4.86319129e+02
 -4.86701225e-01 -7.71312107e-02 -9.99044338e-02]
153
[ 2.31309298e+00  6.68880455e+00  1.18114966e+00 -3.18452153e+02
 -2.32368248e+00 -2.17069601e-02 -5.00285699e-02]
154
[ 1.79500891e+00  4.33689840e+00  6.14440705e+00  3.96512910e+02
  1.79551125e+00 -3.66162496e-02 -5.64213089e-02]
155
[ 4.52205882e-01 -2.49264706e+00  5.67121994

218
[ 1.78030303e+00 -2.88920455e+00  3.66501613e-01 -2.46019488e+02
 -1.53707641e+00  2.09508150e-02  1.13966701e-03]
219
[ 4.37500000e-01 -9.71875000e+00 -5.21078344e+00 -4.82231550e+02
 -2.52628913e+00  8.26829657e-03  2.05625804e-03]
220
[ 4.77272727e-01 -6.07102273e+00 -1.19533322e+00 -4.78998950e+02
 -4.33626841e+00  1.53922383e-02 -1.64597823e-02]
221
[ 0.00000000e+00  3.12500000e-01  3.45380666e-01 -1.42799263e+02
 -1.91200934e+00 -1.56828707e-02 -3.76308404e-02]
222
[ 5.44117647e-01  1.27389706e+00 -9.13596775e-01 -1.20522161e+03
 -8.67043030e-01  4.49248206e-02  2.29285225e-02]
223
[-7.20588235e-01 -7.16727941e+00 -7.47042494e+00 -1.33174118e+03
 -5.44539507e+00 -1.60516763e-02 -6.03840991e-02]
224
[7.50000000e-01 8.62132353e-01 4.58087979e+00 6.04509508e+02
 1.53463404e+00 2.95082309e-02 2.11908337e-02]
225
[ 1.12500000e+00 -3.75000000e+00 -1.61492719e+00 -1.14961651e+02
 -3.18522548e+00  3.39189830e-02  1.24176373e-02]
226
[-2.19696970e-01 -5.67708333e+00 -4.63173369e+00 -3

289
[-1.16969697e+00  9.53333333e+00  4.59532799e+00  6.44589091e+02
  2.78050038e+00 -6.48013836e-03  9.73358742e-03]
290
[9.87500000e-01 1.02729167e+01 4.67276730e+00 5.56304917e+02
 1.64648140e+00 3.31651827e-03 1.32341493e-02]
291
[-6.13793103e-01  5.28045977e+00  2.00579442e+00 -1.58527632e+02
  2.11795827e+00  2.86675252e-04 -4.57372307e-03]
292
[1.31612903e+00 1.45763441e+01 8.58536316e+00 1.95315698e+03
 2.86728407e+00 4.42183263e-02 4.70570476e-02]
293
[-4.37500000e-02  9.08541667e+00  5.16829288e+00  5.90875402e+02
  2.98169781e+00 -2.68163608e-03  1.11583917e-02]
294
[ 1.23333333e+00  1.91666667e+01  1.68151499e+01  1.81024966e+03
  6.94328321e+00 -1.58487986e-02  1.54111160e-03]
295
[-1.09090909e-01  6.20000000e+00  2.89802134e+00 -2.33450743e+02
  2.97319647e+00  6.16908157e-03  1.58028548e-02]
296
[ 1.33125000e+00  9.21041667e+00  2.94804955e+00 -1.69021441e+02
  3.98373426e+00 -4.37910985e-02  2.29432541e-04]
297
[ 2.54545455e-01  8.06060606e-01  4.47445866e+00  8.617976

...

1980
[ 1.48768939e+00 -3.69223485e+00 -2.09746002e+00 -9.11755425e+01
  3.17684772e-01  3.23062084e-02  6.54925502e-02]
1981
[-9.37500000e-01 -5.03125000e+00 -1.28174393e+01 -1.13097879e+03
 -2.98418146e+00 -3.91635186e-02 -3.51313216e-03]
1982
[-8.15340909e-01 -2.78314394e+00 -3.13211812e+00 -4.88683659e+01
 -4.55888493e-01  1.41577076e-03  2.42156688e-02]
1983
[-2.39583333e-01 -6.35890152e+00 -5.15994611e+00 -1.30634126e+02
 -2.34301530e+00  4.38298916e-02  5.21206529e-02]
1984
[-9.63392857e-01  4.09910714e+00  9.61898427e-01 -2.03523112e+02
  3.55718518e+00 -4.53474775e-02  5.48067023e-03]
1985
[ 5.48295455e-01 -1.75284091e+00 -1.18447269e+01 -1.09373859e+03
 -4.18398469e+00 -4.38313980e-02 -1.61648076e-02]
1986
[-1.69412879e+00  1.42897727e+00 -1.89091002e+00 -2.93463779e+02
 -6.33387272e-03  1.83740956e-02  3.21519241e-02]
1987
[ 8.43750000e-01 -3.68750000e+00 -4.13516997e+00 -2.36239634e+02
 -8.12063255e-01  2.10417989e-02  4.19461326e-02]
1988
[-2.17897727e+00 -1.05104167e+01 -1

2050
[-2.45108225e+00  7.79134199e+00  3.05935844e+00 -1.12347570e+02
  3.23950041e+00 -7.76536859e-02 -6.00118800e-02]
2051
[-9.39393939e-01  1.93939394e+00 -9.74726686e+00 -1.00256305e+03
 -4.50166946e+00 -7.61376065e-02 -8.16573579e-02]
2052
[-3.18181818e+00  5.12121212e+00  2.06550001e-01 -2.02288236e+02
 -3.24018645e-01 -1.39321128e-02 -3.33406261e-02]
2053
[-6.43939394e-01  4.73484848e-03 -2.03770995e+00 -1.45064091e+02
 -1.12974803e+00 -1.12644095e-02 -2.35464176e-02]
2054
[-3.66666667e+00 -6.81818182e+00 -1.30053251e+01 -1.31739724e+03
 -6.01749820e+00 -5.34816258e-02 -9.50587330e-02]
2055
[-3.30018939e+00  1.75473485e+00 -4.96708256e+00 -1.78934285e+03
 -4.24221910e+00 -4.78754375e-02 -8.51642806e-02]
2056
[-2.65864528e+00  2.37789661e+00 -5.26945677e+00 -1.05834633e+03
 -1.33839220e+00 -7.74275674e-02 -9.99245894e-02]
2057
[-2.87781036e+00  2.71945259e+00 -4.03710914e+00 -8.47841327e+02
 -2.88677662e+00 -4.40723401e-02 -8.12271252e-02]
2058
[-2.66666667e+00 -4.18181818e+00 -9

2119
[-4.79166667e-01  5.67140152e+00 -1.07879663e+01 -2.31703082e+03
 -3.20917962e+00 -8.01574005e-02 -8.33220915e-02]
2120
[ 1.02083333e+00  3.51515152e+00 -8.05509431e+00 -1.48129220e+03
 -4.10378602e+00 -6.32597234e-02 -6.08764769e-02]
2121
[-2.40196078e+00  7.69162210e+00  2.65051620e+00 -4.25132957e+02
  7.93234479e-02 -2.19109833e-02 -2.71534378e-02]
2122
[-8.78787879e-01  8.45454545e+00  2.02393309e+00 -7.57245094e+02
  2.02326001e+00 -4.13546607e-02 -4.74846529e-02]
2123
[ 4.00000000e-01 -3.68484848e+00 -1.15920895e+01 -1.56313240e+03
 -4.28311882e+00 -8.53279989e-02 -6.41673045e-02]
2124
[-9.88095238e-01 -3.37770563e+00 -4.12391325e+00 -6.43449503e+02
 -3.24104341e+00 -7.39703225e-02 -8.40620988e-02]
2125
[ 1.51168831e+00 -5.85194805e+00 -1.28066253e+01 -8.90215477e+02
 -7.74116987e+00  1.51607946e-03 -2.16454779e-02]
2126
[-7.30735931e-01 -2.67012987e+00 -2.85280844e+00 -8.99406666e+01
 -3.56351905e+00  6.37215731e-02  2.66712539e-02]
2127
[  1.80714286  -7.78660714  -5.0970

2189
[7.88856305e-01 9.53763441e+00 8.96821600e+00 4.69555917e+02
 3.13072158e+00 9.40928572e-03 1.38316078e-02]
2190
[ 1.00000000e+00  2.63636364e+00  3.51707229e+00 -1.01819040e+02
  8.47327090e-01 -4.10382267e-02 -1.55424471e-02]
2191
[7.41477273e-01 1.02291667e+01 1.35731663e+01 2.06106732e+02
 4.66434601e+00 1.30373680e-02 2.96533506e-02]
2192
[ 1.33522727e+00  9.26041667e+00  6.55031259e+00  2.02778240e+02
  2.12977159e+00 -8.63671543e-03  2.44652524e-02]
2193
[2.24331551e+00 7.19607843e+00 3.88020079e+00 6.30752816e+02
 1.91765475e+00 5.47181804e-03 3.54670254e-02]
2194
[ 1.46022727e+00  9.82291667e+00 -8.45127228e-01 -1.03909216e+03
  1.47618506e-01 -1.51520916e-02 -1.63525586e-03]
2195
[ 2.96022727e+00  7.66666667e+00  1.88774474e+00 -2.03353539e+02
 -7.46987894e-01  1.74558557e-03  2.08103587e-02]
2196
[-4.62566845e-01  1.18431373e+01  1.25933553e+01  8.52805704e+02
  3.43612157e+00  4.30943257e-02  5.45333978e-02]
2197
[1.06060606e+00 1.26060606e+01 1.19667721e+01 5.20693567

2259
[-2.70588235e+00  4.64705882e+00  8.71315447e+00  2.22052888e+02
  1.51846682e+00  3.76225076e-02  1.90663724e-02]
2260
[-1.18270945e+00  5.40998217e+00  8.08657135e+00 -1.10059249e+02
  3.46240339e+00  1.81788302e-02 -1.26484268e-03]
2261
[ 9.60784314e-02 -6.72941176e+00 -5.52945121e+00 -9.15946556e+02
 -2.84397544e+00 -2.57945080e-02 -1.79474942e-02]
2262
[-1.29201681 -6.42226891  1.93872502  3.73634159 -1.80190003 -0.01443683
 -0.03784229]
2263
[ 1.50000000e+00 -2.15625000e+00  2.73287197e+00  8.35738624e+02
 -8.94606400e-01  1.68976771e-02  2.24456146e-02]
2264
[-1.92279412e+00  2.02022059e+00  1.34384825e+01  1.89189787e+03
  3.28850307e+00  5.82464172e-02  5.61686537e-02]
2265
[-3.99621212e-01  2.78314394e+00  1.28118994e+01  1.55978573e+03
  5.23243963e+00  3.88027398e-02  3.58374386e-02]
2266
[ 8.79166667e-01 -9.35625000e+00 -8.04123197e-01  7.53898423e+02
 -1.07393920e+00 -5.17059836e-03  1.91547870e-02]
2267
[-5.08928571e-01 -9.04910714e+00  6.66405303e+00  1.67358132e+0

In [19]:
#create training data containing season averages differneces for both teams in all possible matchups
TrainX = np.zeros(shape = (len(X),14))
Feat = ['Score','Blk','OffRt','NetRating','TrueShoot%','TotalR%','AvgDR%']
for i in range(len(X)):
    year = int(SIDs['Season'][i])
    if year == 2016 or year == 2017 or year == 2018:
        Team1 = int(SIDs['WTeamID'][i])
        Team2 = int(SIDs['LTeamID'][i])
        found1 = False
        found2 = False
        for j in range(len(TeamStats['Season'])):
            if int(TeamStats['Season'][j]) == year and int(TeamStats['ID'][j]) == Team1:
                found1 = True
                for k in range(len(Feat)):
                    TrainX[i][k] = TeamStats[Feat[k]][j]
            if int(TeamStats['Season'][j]) == year and int(TeamStats['ID'][j]) == Team2:
                found2 = True
                for k in range(len(Feat)):
                    TrainX[i][k+7] = TeamStats[Feat[k]][j]
            if found1 and found2:
                break;
        print(i)

65872
65873
65874
65875
65876
65877
65878
65879
65880
65881
65882
65883
65884
65885
65886
65887
65888
65889
65890
65891
65892
65893
65894
65895
65896
65897
65898
65899
65900
65901
65902
65903
65904
65905
65906
65907
65908
65909
65910
65911
65912
65913
65914
65915
65916
65917
65918
65919
65920
65921
65922
65923
65924
65925
65926
65927
65928
65929
65930
65931
65932
65933
65934
65935
65936
65937
65938
65939
65940
65941
65942
65943
65944
65945
65946
65947
65948
65949
65950
65951
65952
65953
65954
65955
65956
65957
65958
65959
65960
65961
65962
65963
65964
65965
65966
65967
65968
65969
65970
65971
65972
65973
65974
65975
65976
65977
65978
65979
65980
65981
65982
65983
65984
65985
65986
65987
65988
65989
65990
65991
65992
65993
65994
65995
65996
65997
65998
65999
66000
66001
66002
66003
66004
66005
66006
66007
66008
66009
66010
66011
66012
66013
66014
66015
66016
66017
66018
66019
66020
66021
66022
66023
66024
66025
66026
66027
66028
66029
66030
66031
66032
66033
66034
66035
66036
66037
6603

67240
67241
67242
67243
67244
67245
67246
67247
67248
67249
67250
67251
67252
67253
67254
67255
67256
67257
67258
67259
67260
67261
67262
67263
67264
67265
67266
67267
67268
67269
67270
67271
67272
67273
67274
67275
67276
67277
67278
67279
67280
67281
67282
67283
67284
67285
67286
67287
67288
67289
67290
67291
67292
67293
67294
67295
67296
67297
67298
67299
67300
67301
67302
67303
67304
67305
67306
67307
67308
67309
67310
67311
67312
67313
67314
67315
67316
67317
67318
67319
67320
67321
67322
67323
67324
67325
67326
67327
67328
67329
67330
67331
67332
67333
67334
67335
67336
67337
67338
67339
67340
67341
67342
67343
67344
67345
67346
67347
67348
67349
67350
67351
67352
67353
67354
67355
67356
67357
67358
67359
67360
67361
67362
67363
67364
67365
67366
67367
67368
67369
67370
67371
67372
67373
67374
67375
67376
67377
67378
67379
67380
67381
67382
67383
67384
67385
67386
67387
67388
67389
67390
67391
67392
67393
67394
67395
67396
67397
67398
67399
67400
67401
67402
67403
67404
67405
6740

68606
68607
68608
68609
68610
68611
68612
68613
68614
68615
68616
68617
68618
68619
68620
68621
68622
68623
68624
68625
68626
68627
68628
68629
68630
68631
68632
68633
68634
68635
68636
68637
68638
68639
68640
68641
68642
68643
68644
68645
68646
68647
68648
68649
68650
68651
68652
68653
68654
68655
68656
68657
68658
68659
68660
68661
68662
68663
68664
68665
68666
68667
68668
68669
68670
68671
68672
68673
68674
68675
68676
68677
68678
68679
68680
68681
68682
68683
68684
68685
68686
68687
68688
68689
68690
68691
68692
68693
68694
68695
68696
68697
68698
68699
68700
68701
68702
68703
68704
68705
68706
68707
68708
68709
68710
68711
68712
68713
68714
68715
68716
68717
68718
68719
68720
68721
68722
68723
68724
68725
68726
68727
68728
68729
68730
68731
68732
68733
68734
68735
68736
68737
68738
68739
68740
68741
68742
68743
68744
68745
68746
68747
68748
68749
68750
68751
68752
68753
68754
68755
68756
68757
68758
68759
68760
68761
68762
68763
68764
68765
68766
68767
68768
68769
68770
68771
6877

69974
69975
69976
69977
69978
69979
69980
69981
69982
69983
69984
69985
69986
69987
69988
69989
69990
69991
69992
69993
69994
69995
69996
69997
69998
69999
70000
70001
70002
70003
70004
70005
70006
70007
70008
70009
70010
70011
70012
70013
70014
70015
70016
70017
70018
70019
70020
70021
70022
70023
70024
70025
70026
70027
70028
70029
70030
70031
70032
70033
70034
70035
70036
70037
70038
70039
70040
70041
70042
70043
70044
70045
70046
70047
70048
70049
70050
70051
70052
70053
70054
70055
70056
70057
70058
70059
70060
70061
70062
70063
70064
70065
70066
70067
70068
70069
70070
70071
70072
70073
70074
70075
70076
70077
70078
70079
70080
70081
70082
70083
70084
70085
70086
70087
70088
70089
70090
70091
70092
70093
70094
70095
70096
70097
70098
70099
70100
70101
70102
70103
70104
70105
70106
70107
70108
70109
70110
70111
70112
70113
70114
70115
70116
70117
70118
70119
70120
70121
70122
70123
70124
70125
70126
70127
70128
70129
70130
70131
70132
70133
70134
70135
70136
70137
70138
70139
7014

71341
71342
71343
71344
71345
71346
71347
71348
71349
71350
71351
71352
71353
71354
71355
71356
71357
71358
71359
71360
71361
71362
71363
71364
71365
71366
71367
71368
71369
71370
71371
71372
71373
71374
71375
71376
71377
71378
71379
71380
71381
71382
71383
71384
71385
71386
71387
71388
71389
71390
71391
71392
71393
71394
71395
71396
71397
71398
71399
71400
71401
71402
71403
71404
71405
71406
71407
71408
71409
71410
71411
71412
71413
71414
71415
71416
71417
71418
71419
71420
71421
71422
71423
71424
71425
71426
71427
71428
71429
71430
71431
71432
71433
71434
71435
71436
71437
71438
71439
71440
71441
71442
71443
71444
71445
71446
71447
71448
71449
71450
71451
71452
71453
71454
71455
71456
71457
71458
71459
71460
71461
71462
71463
71464
71465
71466
71467
71468
71469
71470
71471
71472
71473
71474
71475
71476
71477
71478
71479
71480
71481
71482
71483
71484
71485
71486
71487
71488
71489
71490
71491
71492
71493
71494
71495
71496
71497
71498
71499
71500
71501
71502
71503
71504
71505
71506
7150

...

75440
75441
75442
75443
75444
75445
75446
75447
75448
75449
75450
75451
75452
75453
75454
75455
75456
75457
75458
75459
75460
75461
75462
75463
75464
75465
75466
75467
75468
75469
75470
75471
75472
75473
75474
75475
75476
75477
75478
75479
75480
75481
75482
75483
75484
75485
75486
75487
75488
75489
75490
75491
75492
75493
75494
75495
75496
75497
75498
75499
75500
75501
75502
75503
75504
75505
75506
75507
75508
75509
75510
75511
75512
75513
75514
75515
75516
75517
75518
75519
75520
75521
75522
75523
75524
75525
75526
75527
75528
75529
75530
75531
75532
75533
75534
75535
75536
75537
75538
75539
75540
75541
75542
75543
75544
75545
75546
75547
75548
75549
75550
75551
75552
75553
75554
75555
75556
75557
75558
75559
75560
75561
75562
75563
75564
75565
75566
75567
75568
75569
75570
75571
75572
75573
75574
75575
75576
75577
75578
75579
75580
75581
75582
75583
75584
75585
75586
75587
75588
75589
75590
75591
75592
75593
75594
75595
75596
75597
75598
75599
75600
75601
75602
75603
75604
75605
7560

76806
76807
76808
76809
76810
76811
76812
76813
76814
76815
76816
76817
76818
76819
76820
76821
76822
76823
76824
76825
76826
76827
76828
76829
76830
76831
76832
76833
76834
76835
76836
76837
76838
76839
76840
76841
76842
76843
76844
76845
76846
76847
76848
76849
76850
76851
76852
76853
76854
76855
76856
76857
76858
76859
76860
76861
76862
76863
76864
76865
76866
76867
76868
76869
76870
76871
76872
76873
76874
76875
76876
76877
76878
76879
76880
76881
76882
76883
76884
76885
76886
76887
76888
76889
76890
76891
76892
76893
76894
76895
76896
76897
76898
76899
76900
76901
76902
76903
76904
76905
76906
76907
76908
76909
76910
76911
76912
76913
76914
76915
76916
76917
76918
76919
76920
76921
76922
76923
76924
76925
76926
76927
76928
76929
76930
76931
76932
76933
76934
76935
76936
76937
76938
76939
76940
76941
76942
76943
76944
76945
76946
76947
76948
76949
76950
76951
76952
76953
76954
76955
76956
76957
76958
76959
76960
76961
76962
76963
76964
76965
76966
76967
76968
76969
76970
76971
7697

78173
78174
78175
78176
78177
78178
78179
78180
78181
78182
78183
78184
78185
78186
78187
78188
78189
78190
78191
78192
78193
78194
78195
78196
78197
78198
78199
78200
78201
78202
78203
78204
78205
78206
78207
78208
78209
78210
78211
78212
78213
78214
78215
78216
78217
78218
78219
78220
78221
78222
78223
78224
78225
78226
78227
78228
78229
78230
78231
78232
78233
78234
78235
78236
78237
78238
78239
78240
78241
78242
78243
78244
78245
78246
78247
78248
78249
78250
78251
78252
78253
78254
78255
78256
78257
78258
78259
78260
78261
78262
78263
78264
78265
78266
78267
78268
78269
78270
78271
78272
78273
78274
78275
78276
78277
78278
78279
78280
78281
78282
78283
78284
78285
78286
78287
78288
78289
78290
78291
78292
78293
78294
78295
78296
78297
78298
78299
78300
78301
78302
78303
78304
78305
78306
78307
78308
78309
78310
78311
78312
78313
78314
78315
78316
78317
78318
78319
78320
78321
78322
78323
78324
78325
78326
78327
78328
78329
78330
78331
78332
78333
78334
78335
78336
78337
78338
7833

79541
79542
79543
79544
79545
79546
79547
79548
79549
79550
79551
79552
79553
79554
79555
79556
79557
79558
79559
79560
79561
79562
79563
79564
79565
79566
79567
79568
79569
79570
79571
79572
79573
79574
79575
79576
79577
79578
79579
79580
79581
79582
79583
79584
79585
79586
79587
79588
79589
79590
79591
79592
79593
79594
79595
79596
79597
79598
79599
79600
79601
79602
79603
79604
79605
79606
79607
79608
79609
79610
79611
79612
79613
79614
79615
79616
79617
79618
79619
79620
79621
79622
79623
79624
79625
79626
79627
79628
79629
79630
79631
79632
79633
79634
79635
79636
79637
79638
79639
79640
79641
79642
79643
79644
79645
79646
79647
79648
79649
79650
79651
79652
79653
79654
79655
79656
79657
79658
79659
79660
79661
79662
79663
79664
79665
79666
79667
79668
79669
79670
79671
79672
79673
79674
79675
79676
79677
79678
79679
79680
79681
79682
79683
79684
79685
79686
79687
79688
79689
79690
79691
79692
79693
79694
79695
79696
79697
79698
79699
79700
79701
79702
79703
79704
79705
79706
7970

80910
80911
80912
80913
80914
80915
80916
80917
80918
80919
80920
80921
80922
80923
80924
80925
80926
80927
80928
80929
80930
80931
80932
80933
80934
80935
80936
80937
80938
80939
80940
80941
80942
80943
80944
80945
80946
80947
80948
80949
80950
80951
80952
80953
80954
80955
80956
80957
80958
80959
80960
80961
80962
80963
80964
80965
80966
80967
80968
80969
80970
80971
80972
80973
80974
80975
80976
80977
80978
80979
80980
80981
80982
80983
80984
80985
80986
80987
80988
80989
80990
80991
80992
80993
80994
80995
80996
80997
80998
80999
81000
81001
81002
81003
81004
81005
81006
81007
81008
81009
81010
81011
81012
81013
81014
81015
81016
81017
81018
81019
81020
81021
81022
81023
81024
81025
81026
81027
81028
81029
81030
81031
81032
81033
81034
81035
81036
81037
81038
81039
81040
81041
81042
81043
81044
81045
81046
81047
81048
81049
81050
81051
81052
81053
81054
81055
81056
81057
81058
81059
81060
81061
81062
81063
81064
81065
81066
81067
81068
81069
81070
81071
81072
81073
81074
81075
8107

In [26]:
FX = np.zeros(shape=(len(TrainX),7))
for i in range(len(TrainX)):
    for j in range(0,7):
        FX[i][j] = TrainX[i][j]-TrainX[i][j+7]

In [28]:
#Extract needed training data for predicting 2019
f1 = True
for i in range(len(SIDs['Season'])):
    if SIDs['Season'][i] == 2016 and f1:
        start = i
    elif SIDs['Season'][i] == 2018:
        end = i
X_2019 = FX[start:end+1]
Y_2019 = TrainY[start:end+1]


In [32]:
#fit training data to linear regression model
#create predictions for winning margins for all possible matchups in 2019
#generate winning probabilities of teams with lower id by adding winning margin multiplied by a weight of 0.029 to 0.5
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_2019,Y_2019)
pred = model.predict(matchups)
sampSub['Pred'] = 0.5 + pred*0.029

In [34]:
#Clip the predictions so that no predction is too confident
for i in range(len(sampSub['Pred'])):
    if sampSub['Pred'][i] < 0.10:
        sampSub['Pred'][i] = 0.15
    elif sampSub['Pred'][i] > 0.90:
        sampSub['Pred'][i] = 0.85

/Users/fraserthomson/ml/env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/fraserthomson/ml/env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [35]:
sampSub

,ID,Pred
0,2019_1101_1113,0.417578
1,2019_1101_1120,0.247078
2,2019_1101_1124,0.556533
3,2019_1101_1125,0.150000
4,2019_1101_1133,0.833904
5,2019_1101_1138,0.150000
6,2019_1101_1153,0.419786
7,2019_1101_1159,0.427337
8,2019_1101_1181,0.150000
9,2019_1101_1192,0.527561


In [37]:
sampSub.to_csv('LR1(clipped).csv',index=False)